In [137]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import cohen_kappa_score
from imblearn.over_sampling import SMOTE
from collections import Counter

In [139]:
with open ('yeast1.dat') as f:
     lines = [line.rstrip('\n')for line in f]

In [90]:
del lines[0:13]
data_string_list = []
for i in range(len(lines)):
    data_string_list.append(lines[i].split(','))

In [92]:
labels=[]
for i in range(len(data_string_list)):
    if data_string_list[i][-1] == ' negative':
        labels.append(0)
    else:
        labels.append(1)
labels=np.array(labels)

In [94]:
np.unique(labels)

array([0, 1])

In [96]:
data_string_list=np.array(data_string_list)
data_string_list=data_string_list[:,:-1]

In [98]:
data_list=[]
for i in range(len(data_string_list)):
    row_list=[]
    for j in range(len(data_string_list[0])):
        f= float(data_string_list[i][j])
        row_list.append(f)
    data_list.append(row_list)
data = np.array(data_list)

In [100]:
data.shape

(1484, 8)

### K-fold:

In [123]:
def get_folds(data,labels,k):
  np.random.seed(42)
  shuffle=np.random.permutation(len(data))
  data=data[shuffle]
  labels=labels[shuffle]

  block_length=int(len(data)/k)
  data_folds=[]
  label_folds=[]
  for i in range(k):
    if i==(k-1):
      data_fold=data[i*block_length:]
      data_folds.append(data_fold)
      label_fold=labels[i*block_length:]
      label_folds.append(label_fold)
    else:
      data_fold=data[i*block_length:block_length*(i+1)]
      data_folds.append(data_fold)
      label_fold=labels[i*block_length:block_length*(i+1)]
      label_folds.append(label_fold)
  return np.array(data_folds,dtype=object),np.array(label_folds,dtype=object)

In [146]:
get_folds(data,labels,5)

(array([array([[0.59, 0.6 , 0.49, ..., 0.  , 0.53, 0.31],
               [0.45, 0.56, 0.51, ..., 0.  , 0.54, 0.22],
               [0.57, 0.5 , 0.4 , ..., 0.  , 0.47, 0.22],
               ...,
               [0.57, 0.36, 0.51, ..., 0.  , 0.54, 0.25],
               [0.33, 0.37, 0.54, ..., 0.  , 0.53, 0.27],
               [0.59, 0.67, 0.54, ..., 0.  , 0.48, 0.6 ]]),
        array([[0.42, 0.5 , 0.64, ..., 0.  , 0.49, 0.22],
               [0.31, 0.45, 0.5 , ..., 0.  , 0.49, 0.22],
               [0.5 , 0.45, 0.48, ..., 0.  , 0.5 , 0.26],
               ...,
               [0.48, 0.45, 0.59, ..., 0.  , 0.55, 0.3 ],
               [0.35, 0.21, 1.  , ..., 0.  , 0.13, 0.01],
               [0.55, 0.44, 0.5 , ..., 0.  , 0.55, 0.22]]),
        array([[0.57, 0.47, 0.48, ..., 0.  , 0.52, 0.26],
               [0.45, 0.48, 0.63, ..., 0.  , 0.48, 0.22],
               [0.43, 0.4 , 0.6 , ..., 0.  , 0.53, 0.39],
               ...,
               [0.54, 0.35, 0.55, ..., 0.  , 0.3 , 0.22],
        

In [125]:
def divide_data_into_k_folds(data,labels,k):
  data_folds_array,label_folds_array=get_folds(data,labels,k)
  all_data_folds_list=[]
  all_labels_folds_list=[]
  for i in range(k):
    data_fold_k=[np.concatenate(data_folds_array[np.where(np.array(list(range(k)))!=i)[0]]), data_folds_array[i]]
    all_data_folds_list.append(data_fold_k)

    label_fold_k=[np.concatenate(label_folds_array[np.where(np.array(list(range(k)))!=i)[0]]), label_folds_array[i]]
    all_labels_folds_list.append(label_fold_k)

  return all_data_folds_list, all_labels_folds_list

In [127]:
all_data_folds_list, all_labels_folds_list=divide_data_into_k_folds(data,labels,5)

In [14]:
X_train,X_test=all_data_folds_list[0][0],all_data_folds_list[0][1]

In [15]:
X_train.shape

(1188, 8)

In [16]:
y_train,y_test=all_labels_folds_list[0][0],all_labels_folds_list[0][1]

In [17]:
X_train_minority=[]
for i in np.where(y_train==1):
    a=X_train[i]
    X_train_minority.append(a)

In [18]:
X_train_minority_class=np.array(X_train_minority[0])

In [19]:
X_train_minority_class.shape

(344, 8)

In [20]:
X_train_majority=[]
for i in np.where(y_train==0):
    a=X_train[i]
    X_train_majority.append(a)

In [21]:
X_train_majority_class=np.array(X_train_majority[0])

In [22]:
X_train_majority_class.shape

(844, 8)

### Kernel Density Functon:

In [24]:
def k(H,x,d):
  H_inv = np.linalg.inv(H)
  x_tras=np.matrix.transpose(x)
  h= np.linalg.det(H)
  m=np.matmul(x_tras,H_inv)
  m1=np.matmul(m,x)
  e=np.exp(-0.5*m1)
  n=2*(np.pi)**(-d/2)
  v=n*(h)**(-1/2)*(e)
  return v

In [25]:
def density_gen(X, X_minority):
    density_list=[]
    for j in range(len(X)):
        s=0
        for i in range(len(X_minority)):
            xi=X_minority[i]
            dif=X[j]-xi
            s=s+k(np.identity(X_minority.shape[1]), dif, X_minority.shape[1])
        density_list.append(s/len(X_minority))
    return np.array(density_list)

In [26]:
unique_labels,counts=np.unique(y_train, return_counts=True)

In [27]:
def metropolis_hastings(target_density, X_minority, size=100):
    np.random.seed(43)
    burnin_size = 10000
    size += burnin_size
    x0 =X_minority[np.random.choice(list(range(len(X_train_minority_class))),1)]
    xt = x0
    samples = []
    for i in tqdm(range(size)):
        xt_candidate = np.array([np.random.multivariate_normal(xt[0], 0.2*np.eye(X_minority.shape[1]))])
        accept_odds = (target_density(xt_candidate,X_minority))/(target_density(xt,X_minority))
        if np.random.uniform(0, 1) < accept_odds:
            xt = xt_candidate
        samples.append(xt)
    samples = np.array(samples[burnin_size:])
    samples = np.reshape(samples, [samples.shape[0], X_minority.shape[1]])
    return samples

In [28]:
import smote_variants as sv

oversampler_SMOTE=sv.SMOTE()
oversampler_polynome_fit = sv.polynom_fit_SMOTE_star()
oversampler_ProWSyn= sv.ProWSyn()
oversampler_SMOTE_IPF= sv.SMOTE_IPF()
oversampler_Lee= sv.Lee()


In [29]:
oversampler_object_list = [oversampler_SMOTE,oversampler_polynome_fit,oversampler_ProWSyn, oversampler_Lee, metropolis_hastings]

In [30]:
oversampler_name_list = ['SMOTE','polynome_fit_SMOTE','ProWSyn', 'Lee', 'Metropolis_hastings']

In [31]:
def get_imb_oversampling(X_train,y_train,X_test,y_test,oversampler_index,classifier,visualize=True):
  ## MH is in the end of the list oversampler_object_list
  if oversampler_index!=len(oversampler_object_list)-1:
    X_samp, y_samp = oversampler_object_list[oversampler_index].sample(X_train, y_train)

  else:
    X_train_minority=[]
    for i in np.where(y_train==1):
      a=X_train[i]
      X_train_minority.append(a)

    X_train_majority=[]
    for i in np.where(y_train==0):
      a=X_train[i]
      X_train_majority.append(a)
    X_train_minority_class=np.array(X_train_minority[0])
    X_train_majority_class=np.array(X_train_majority[0])

    unique_labels,counts=np.unique(y_train, return_counts=True)

    synthetic_minority_data = metropolis_hastings(density_gen, X_train_minority_class, size=counts[0]-counts[1])
    balanced_minority=np.concatenate([X_train_minority_class, synthetic_minority_data])
    balanced_data=np.concatenate([X_train_majority_class,balanced_minority])
    balanced_labels_for_classification=np.concatenate([np.zeros(len(X_train_majority_class)),np.ones(len(balanced_minority))])
    X_samp=balanced_data
    y_samp=balanced_labels_for_classification

  classifier.fit(X_samp,y_samp)
  y_pred=classifier.predict(X_test)
  cm=confusion_matrix(y_test, y_pred)
  acc=accuracy_score(y_test, y_pred)
  b_acc=balanced_accuracy_score(y_test, y_pred)
  gm=geometric_mean_score(y_test, y_pred,average='micro')
  f1=f1_score(y_test, y_pred,average='micro')
  ck=cohen_kappa_score(y_test, y_pred)

  if visualize==True:
    scaler = StandardScaler()
    scaler.fit(X_samp)
    scaled_X_samp = scaler.transform(X_samp)
    pca = PCA(n_components =2)
    X_samp_pca=pca.fit_transform(scaled_X_samp)

    plt.figure(figsize=(8,6))
    color=["r","b"]
    color_counter=0
    for i in [-1,1]:
      plt.scatter(X_samp_pca[:,0][np.where(y_samp==i)],X_samp_pca[:,1][np.where(y_samp==i)], s=10, c=color[color_counter])
      color_counter=color_counter+1
    plt.legend(['Majority','Manority'],fontsize=10, loc='lower right')
    plt.xlabel("Principal Component 1",fontsize=12)
    plt.ylabel("Principlal Component 2 ",fontsize=12)
    plt.title("PCA plot of majority class + minority class using"+""+ oversampler_name_list[oversampler_index]+"technique",fontsize=14)


  return [cm, acc,b_acc,gm, f1, ck]

In [32]:
def calc_avg_performance(performance_index,results_all_folds):
  ## performance index takes the index of the performance measure foe which we want to average over folds
  ## results_all_folds are the results of a particular oversampler run over all folds recording all performance measures
  s=0
  for result in results_all_folds:
    s=s+result[performance_index]
  avg=s/len(results_all_folds)

  s=0
  for result in results_all_folds:
    s=s+(result[performance_index]-avg)**2
  std=np.sqrt(s/len(results_all_folds))
  return avg, std

In [33]:
clf = DecisionTreeClassifier(random_state=0)

In [34]:
perfromance_measures=['accuracy','balanced accuracy','g-mean','f1-score','cohen_kappa']
def get_results_all_folds(oversampler_index,classifier, visualize=False, get_detailed_results=False):

  results_all_folds=[]
  for i in range(len(all_data_folds_list)):
    X_train,X_test=all_data_folds_list[i][0],all_data_folds_list[i][1]
    y_train,y_test=all_labels_folds_list[i][0],all_labels_folds_list[i][1]
    results_single_fold=get_imb_oversampling(X_train,y_train,X_test,y_test,oversampler_index,clf,visualize=visualize)
    results_all_folds.append(results_single_fold)

  performance_summary_oversampler_classifer={}
  for performance_index in range(1,6):
    avg_std=calc_avg_performance(performance_index,results_all_folds)
    performance_summary_oversampler_classifer[perfromance_measures[performance_index-1]]=avg_std

  if get_detailed_results==True:
    return results_all_folds, performance_summary_oversampler_classifer
  else:
    return performance_summary_oversampler_classifer

In [35]:
get_results_all_folds(0,clf)

2024-08-22 10:36:23,144:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2024-08-22 10:36:23,146:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2024-08-22 10:36:23,148:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2024-08-22 10:36:23,150:INFO:SMOTE: simplex sampling with n_dim 2
2024-08-22 10:36:23,167:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2024-08-22 10:36:23,167:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2024-08-22 10:36:23,167:INFO:NearestNeighborsW

{'accuracy': (0.6839909909909909, 0.019997597453291837),
 'balanced accuracy': (0.6394484520634182, 0.024507883259942354),
 'g-mean': (0.6839909909909909, 0.019997597453291837),
 'f1-score': (0.6839909909909909, 0.019997597453291837),
 'cohen_kappa': (0.2661909135948061, 0.04923675271858221)}

In [36]:
summary_performace_for_all_oversampling_algorithms_all_folds=[]
for i in range(len(oversampler_object_list)):
  summary_performace_for_all_oversampling_algorithms_all_folds.append(get_results_all_folds(i,clf))

2024-08-22 10:36:23,270:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2024-08-22 10:36:23,272:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2024-08-22 10:36:23,273:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2024-08-22 10:36:23,278:INFO:SMOTE: simplex sampling with n_dim 2
2024-08-22 10:36:23,299:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2024-08-22 10:36:23,301:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2024-08-22 10:36:23,303:INFO:NearestNeighborsW

  0%|          | 0/10500 [00:00<?, ?it/s]

  0%|          | 0/10494 [00:00<?, ?it/s]

  0%|          | 0/10512 [00:00<?, ?it/s]

  0%|          | 0/10510 [00:00<?, ?it/s]

  0%|          | 0/10488 [00:00<?, ?it/s]

In [37]:
summary_performace_for_all_oversampling_algorithms_all_folds_list=np.array([[[round(i,3) for i in summary_performace_for_all_oversampling_algorithms_all_folds[oversampler_index][key]] for key in summary_performace_for_all_oversampling_algorithms_all_folds[0].keys()] for oversampler_index in range(len(oversampler_name_list))])

In [38]:
oversamplers_result_summary_list=[]
for i in range(summary_performace_for_all_oversampling_algorithms_all_folds_list.shape[0]):
  performance_results_summary_list=[]
  for j in range(summary_performace_for_all_oversampling_algorithms_all_folds_list.shape[1]):
    performance_results_summary_list.append(str(summary_performace_for_all_oversampling_algorithms_all_folds_list[i][j][0])+"\u00B1"+str(summary_performace_for_all_oversampling_algorithms_all_folds_list[i][j][1]))
  oversamplers_result_summary_list.append(performance_results_summary_list)
oversamplers_result_summary_list=np.array(oversamplers_result_summary_list)

In [39]:
performance_results_summary_list

['0.708±0.016', '0.653±0.025', '0.708±0.016', '0.708±0.016', '0.301±0.051']

In [40]:
oversamplers_result_summary_list

array([['0.699±0.017', '0.656±0.015', '0.699±0.017', '0.699±0.017',
        '0.299±0.035'],
       ['0.704±0.027', '0.64±0.035', '0.704±0.027', '0.704±0.027',
        '0.279±0.069'],
       ['0.718±0.023', '0.679±0.031', '0.718±0.023', '0.718±0.023',
        '0.342±0.056'],
       ['0.703±0.029', '0.654±0.034', '0.703±0.029', '0.703±0.029',
        '0.299±0.071'],
       ['0.708±0.016', '0.653±0.025', '0.708±0.016', '0.708±0.016',
        '0.301±0.051']], dtype='<U11')

In [41]:
df=pd.DataFrame(oversamplers_result_summary_list,columns=perfromance_measures,index=oversampler_name_list)

In [42]:
df

,accuracy,balanced accuracy,g-mean,f1-score,cohen_kappa
SMOTE,0.699±0.017,0.656±0.015,0.699±0.017,0.699±0.017,0.299±0.035
polynome_fit_SMOTE,0.704±0.027,0.64±0.035,0.704±0.027,0.704±0.027,0.279±0.069
ProWSyn,0.718±0.023,0.679±0.031,0.718±0.023,0.718±0.023,0.342±0.056
Lee,0.703±0.029,0.654±0.034,0.703±0.029,0.703±0.029,0.299±0.071
Metropolis_hastings,0.708±0.016,0.653±0.025,0.708±0.016,0.708±0.016,0.301±0.051


In [82]:
with pd.ExcelWriter('Decision_tree_clf_result_for_d2.xlsx') as excel_writer:
  df.to_excel(excel_writer,sheet_name='sheet_1')